# Experiment with Fashion MNIST
## Centralized learning for baseline

In [ ]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import pickle
import time
import datetime

import tensorflow as tf
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(0)
import tensorflow_hub as hub
import numpy as np
import pandas as pd

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
labelnames = ['T-shirt', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
labelnames

In [ ]:
dataset_root = os.path.abspath(os.path.expanduser('dataset-fashionmnist'))
dataset_root = os.path.join(dataset_root, 'center')
dataset_root

In [ ]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
target_size = (28, 28, 3)
datasets = dict()
for subset in ['train', 'test']:
    path = os.path.join(dataset_root, subset)
    datasets[subset] = image_generator.flow_from_directory(path, classes=labelnames,
                                                           target_size=target_size[:-1], shuffle=True,
                                                           follow_links=True)
datasets

In [ ]:
model = tf.keras.models.Sequential([
                       tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=target_size),
                       tf.keras.layers.MaxPooling2D((2, 2)),
                       tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
                       tf.keras.layers.MaxPooling2D((2, 2)),
                       tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
                       tf.keras.layers.Flatten(),
                       tf.keras.layers.Dense(128, activation='relu'),
                       tf.keras.layers.Dense(len(labelnames))])
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

In [ ]:
epochs = 200
returns = model.fit(datasets['train'],
                    validation_data=datasets['test'],
                    epochs=epochs)

In [ ]:
today = datetime.datetime.now().strftime('%y%m%d%H%M%S')
with open(f'result-fashionmnist-centralizedlearning-{today}.pkl', 'wb') as f:
    pickle.dump(returns.history, f)